# Sales Forecasting Analysis

## Project Overview
This notebook demonstrates a complete workflow for forecasting retail sales using historical transaction data. We will use **Facebook Prophet**, a robust library for time series forecasting, to predict future sales trends.

### Objectives
1.  **Data Loading & Cleaning**: Prepare the raw dataset for analysis.
2.  **Exploratory Data Analysis (EDA)**: Visualize historical sales trends.
3.  **Modeling**: Train a Prophet model on the aggregated sales data.
4.  **Forecasting**: Predict sales for the next 90 days.
5.  **Advanced Insights**: Analyze monthly trends, top products, and seasonality.
6.  **Export**: Save the forecast results for Power BI dashboarding.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet

# Plotting settings
plt.style.use('seaborn-v0_8-whitegrid')
# Custom professional palette (Deep Blue, Teal, Gold, Coral, Purple)
custom_palette = ['#2C3E50', '#18BC9C', '#F39C12', '#E74C3C', '#8E44AD']
sns.set_palette(custom_palette)
sns.set_context('notebook', font_scale=1.2)
%matplotlib inline

## 1. Data Loading and Cleaning

In [ ]:
# Load the dataset
# Using 'ISO-8859-1' encoding to handle special characters common in retail datasets
try:
    df = pd.read_csv('Dataset.csv', encoding='ISO-8859-1')
except UnicodeDecodeError:
    df = pd.read_csv('Dataset.csv', encoding='cp1252')

# Display first few rows
df.head()

In [ ]:
# Check data info
df.info()

In [ ]:
# Convert ORDERDATE to datetime
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

# Sort by date
df = df.sort_values('ORDERDATE')

# Check date range
print(f"Date Range: {df['ORDERDATE'].min()} to {df['ORDERDATE'].max()}")

## 2. Exploratory Data Analysis (EDA)
We will aggregate sales by date to understand the overall trend.

In [ ]:
# Aggregate sales by Date
daily_sales = df.groupby('ORDERDATE')['SALES'].sum().reset_index()

# Plot Daily Sales
plt.figure(figsize=(15, 6))
plt.plot(daily_sales['ORDERDATE'], daily_sales['SALES'], label='Daily Sales', color='#2C3E50', alpha=0.7)
plt.title('Daily Sales Over Time', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Sales ($)')
plt.legend()
plt.tight_layout()
plt.show()

## 3. Modeling with Prophet
Prophet requires the input dataframe to have two specific columns:
- `ds`: The datestamp column.
- `y`: The numeric metric to forecast.

In [ ]:
# Prepare data for Prophet
prophet_df = daily_sales.rename(columns={'ORDERDATE': 'ds', 'SALES': 'y'})

# Initialize Prophet model
# Tuning for better accuracy:
# 1. Multiplicative seasonality (sales grow with trend)
# 2. US Holidays (Retail impact)
# 3. More flexible trend (changepoint_prior_scale)
m = Prophet(seasonality_mode='multiplicative', 
            daily_seasonality=False, 
            yearly_seasonality=True, 
            weekly_seasonality=True,
            changepoint_prior_scale=0.5,
            seasonality_prior_scale=10.0)

# Add US Holidays
m.add_country_holidays(country_name='US')

# Fit the model
m.fit(prophet_df)

## 4. Forecasting
We will generate a forecast for the next 90 days.

In [ ]:
# Create future dataframe for 90 days
future = m.make_future_dataframe(periods=90)

# Predict
forecast = m.predict(future)

# Display forecast head
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# Visualize the forecast
fig1 = m.plot(forecast)
plt.title('Sales Forecast (Next 90 Days)', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Visualize Components Custom Plots
# Extract components
forecast['day_of_week'] = forecast['ds'].dt.day_name()
forecast['day_num'] = forecast['ds'].dt.dayofweek

# 1. Trend Component
plt.figure(figsize=(15, 6))
sns.lineplot(data=forecast, x='ds', y='trend', color='#2C3E50', linewidth=2)
plt.title('Sales Trend Component', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Trend')
plt.tight_layout()
plt.show()

# 2. Weekly Seasonality Component
# Group by day of week to get the unique weekly effect
weekly_seasonality = forecast.groupby(['day_num', 'day_of_week'])['weekly'].mean().reset_index().sort_values('day_num')

plt.figure(figsize=(10, 6))
sns.barplot(data=weekly_seasonality, x='day_of_week', y='weekly', palette='viridis')
plt.title('Weekly Seasonality Component', fontsize=16)
plt.xlabel('Day of Week')
plt.ylabel('Weekly Effect')
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
plt.tight_layout()
plt.show()

# 3. Yearly Seasonality Component
forecast['day_of_year'] = forecast['ds'].dt.dayofyear
yearly_seasonality = forecast.groupby('day_of_year')['yearly'].mean().reset_index()

plt.figure(figsize=(15, 6))
sns.lineplot(data=yearly_seasonality, x='day_of_year', y='yearly', color='#E74C3C', linewidth=2)
plt.title('Yearly Seasonality Component', fontsize=16)
plt.xlabel('Day of Year')
plt.ylabel('Yearly Effect')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate Projected Growth
# Compare the last forecasted value with the value 90 days prior
future_period = 90
last_forecast_value = forecast['yhat'].iloc[-1]
first_forecast_value = forecast['yhat'].iloc[-future_period]
projected_growth = ((last_forecast_value / first_forecast_value) - 1) * 100

print(f"Projected Sales Growth over next {future_period} days: {projected_growth:.2f}%")

## 5. Advanced Insights for Decision Making
We will derive key business insights to support decision making.

In [ ]:
# 1. Monthly & Yearly Trends
df['Year'] = df['ORDERDATE'].dt.year
df['Month'] = df['ORDERDATE'].dt.month

monthly_sales = df.groupby(['Year', 'Month'])['SALES'].sum().reset_index()
monthly_sales['Date'] = pd.to_datetime(monthly_sales[['Year', 'Month']].assign(DAY=1))

plt.figure(figsize=(15, 6))
sns.lineplot(data=monthly_sales, x='Date', y='SALES', marker='o', color='#18BC9C', linewidth=2.5)
plt.title('Monthly Sales Trend', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.tight_layout()
plt.show()

In [ ]:
# 2. Top Selling Items
top_items = df.groupby('PRODUCTCODE')['SALES'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_items.index, y=top_items.values, palette='viridis')
plt.title('Top 10 Selling Products', fontsize=16)
plt.ylabel('Total Sales ($)')
plt.xlabel('Product Code')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3. Sales by Category (Product Line)
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='SALES', y='PRODUCTLINE', estimator=sum, errorbar=None, palette='mako')
plt.title('Total Sales by Product Line', fontsize=16)
plt.xlabel('Total Sales ($)')
plt.ylabel('Product Line')
plt.tight_layout()
plt.show()

In [ ]:
# 4. Sales by Region (Country)
top_countries = df.groupby('COUNTRY')['SALES'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_countries.values, y=top_countries.index, palette='magma')
plt.title('Top 10 Countries by Sales', fontsize=16)
plt.xlabel('Total Sales ($)')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

In [ ]:
# 5. Seasonality Analysis (Best/Worst Months)
avg_monthly_sales = df.groupby('Month')['SALES'].mean()

plt.figure(figsize=(12, 6))
sns.barplot(x=avg_monthly_sales.index, y=avg_monthly_sales.values, palette='coolwarm')
plt.title('Average Sales by Month (Seasonality)', fontsize=16)
plt.xlabel('Month')
plt.ylabel('Average Sales ($)')
plt.xticks(ticks=range(0, 12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.tight_layout()
plt.show()

## 6. Export Results
We will export the forecast data to a CSV file. This file can be directly imported into Power BI to build the dashboard.

In [ ]:
# Prepare export dataframe
# We'll merge the actuals with the forecast for a complete view
export_df = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()

# Merge with actuals
export_df = export_df.merge(prophet_df, on='ds', how='left')
export_df.rename(columns={'ds': 'Date', 'y': 'Actual_Sales', 'yhat': 'Predicted_Sales', 'yhat_lower': 'Lower_Bound', 'yhat_upper': 'Upper_Bound'}, inplace=True)

# Fill NaN in Actual_Sales with 0 or keep as NaN depending on preference (keeping as NaN for future dates)

# Save to CSV
export_df.to_csv('forecast_results_full.csv', index=False)
print("Exported successfully to 'forecast_results_full.csv'")